In [22]:
import os
import wandb

import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Dense

# global random seed
SEED = 0

In [18]:
# create image generator
image_generator = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rescale=None,  # scaling is done in the model
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    interpolation_order=1,
    dtype=None)

In [19]:
# train
train_generator = image_generator.flow_from_directory(
    directory='data/train',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    seed=SEED
)

# validation
valid_generator = image_generator.flow_from_directory(
    directory='data/valid',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    seed=SEED
)

# test
test_generator = image_generator.flow_from_directory(
    directory='data/test',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    seed=SEED
)

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [16]:
# load pre-trained model
pretrained = load_model('models/EfficientNetB0-525-(224 X 224)- 98.97.h5', custom_objects={"F1_score":"F1_score"})

# view the model
pretrained.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  0           ['rescaling[0][0]']              
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 32  864         ['normalization[0][0]']      

In [24]:
# comile the model
pretrained.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [25]:
# evaluate on training data
pretrained.evaluate(train_generator)

2645/2645 [==============================] - 678s 255ms/step - loss: 0.2121 - accuracy: 0.9987


[0.2121136486530304, 0.9987475872039795]

In [26]:
# evaluate on validation data
pretrained.evaluate(valid_generator)

83/83 [==============================] - 21s 253ms/step - loss: 0.3335 - accuracy: 0.9783


[0.33352071046829224, 0.9782857298851013]

In [27]:
# evaluate on test data
pretrained.evaluate(test_generator)

83/83 [==============================] - 22s 259ms/step - loss: 0.2549 - accuracy: 0.9897


[0.2548711895942688, 0.9897142648696899]

In [ ]:
# # initialize weights and biases
# wandb.init(
#     project='bird classification', entity='heefjones', 
#     config={"architecture":"EffiecientNetB0",
#             "dataset":"Kaggle 525 Bird Classification",
#             "epochs":10,
#             "batch_size": 32
# })

# # finish run
# wandb.finish()